In [1]:
import pandas as pd
import numpy as np
import csv
import glob
import os
from scipy import stats
import math
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import datetime
from datetime import datetime, timedelta
import holidays
import statsmodels.formula.api as smf
pd.options.display.max_rows = 50

pd.options.display.float_format = '{:.2f}'.format

In [2]:
# # Extraneous Analysis
# #Initial Data Cleaning from Model 4
df = pd.read_csv("ben_intraday_data.csv")
df['str_date'] = df['date']
df.drop('date',axis=1)
df['datetime'] = df['date'] + ' ' + df['time']
df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = pd.to_datetime(df['date'])
df.index = pd.DatetimeIndex(df['datetime'])
trading_days = df.date.unique()

# df = df.fillna('')
# df['symbol'] = df['sym_root'] + df['sym_suffix']
# Sector = df.loc[(df.sym_root == 'NFLX') | (df.sym_root == 'PYPL') | (df.sym_root == 'NFLX') | (df.sym_root == 'MSFT')]
# S = Sector.groupby(['datetime','symbol']).sum().groupby('datetime').sum()
# S = S.between_time('09:31:00', '15:59:00').sort_index()

# MSFT = Sector.loc[Sector.symbol == 'MSFT']
# MSFT.index = pd.DatetimeIndex(MSFT['datetime'])
# MSFT = MSFT.between_time('09:31:00', '15:59:00').sort_index()
# MSFT['cum_vol'] = np.nan
# MSFT = MSFT.resample('60s').asfreq()
# MSFT['size'].fillna(0, inplace=True)
# MSFT['date'] = MSFT.index.date
# MSFT['time'] = MSFT.index.time
# MSFT['date'] = pd.to_datetime(MSFT.date) 
# MSFT = MSFT.loc[MSFT.date.isin(rel), :]
# MSFT = MSFT.between_time('09:31:00', '15:59:00')
# S['cum_vol'] = np.nan
# S = S.resample('60s').asfreq()
# S['size'].fillna(0, inplace=True)
# S['date'] = S.index.date
# S['time'] = S.index.time
# S['date'] = pd.to_datetime(S.date) 
# S = S.loc[S.date.isin(trading_days), :]
# S = S.between_time('09:31:00', '15:59:00')
# trading_days = list(MSFT.date.unique())
# S['date'] = pd.to_datetime(S.index) 
# S['date'] = S['date'].dt.date
# S['date'] = pd.to_datetime(S.date) 
# for day in trading_days:
#     S.loc[day==S.date, 'cum_vol'] = S.loc[day== S.date,'size'].cumsum()
#     MSFT.loc[day==MSFT.date, 'cum_vol'] = MSFT.loc[day== MSFT.date,'size'].cumsum()
# for days in trading_days:
#     num = MSFT['cum_vol'][MSFT.date == days].values[-1]
#     num1 = S['cum_vol'][S.date == days].values[-1]
#     S.loc[days==S.date, 'Correction'] = num1
#     MSFT.loc[days==MSFT.date, 'Correction'] = num
# MSFT['CDF'] = MSFT['cum_vol']/MSFT['Correction']
# S['CDF'] = S['cum_vol']/S['Correction']
# x = S.CDF
# y = MSFT.CDF
# np.corrcoef(x,y)

In [3]:
# A rather inefficient way to get the trading days, but it works
# Gives the days until June 1st, 2019 (can be adjusted easily)
K = pd.Series(trading_days)
deys = pd.DataFrame({'Dates':K.values})
rel = deys.loc[inept.Dates < datetime(2019,6,1)].Dates.unique()

In [4]:
# Creates the dataframe, Total, with all of the S&P 500 Stocks 
# Produces a datetime and symbol column
Total = pd.DataFrame()
for day in rel:
    cur = pd.Timestamp(day).strftime("%Y%m%d")
    csvFile = "/Users/bengoldman/Desktop/Junior Year/Data Science Clinic/SP500/SP500_" + str(cur) + ".csv"
    partial = pd.read_csv(csvFile)
    Total = pd.concat([Total,partial])
Total = Total.reset_index(drop = True)
Total['str_date'] = Total['date']
Total.drop('date',axis=1)
Total['datetime'] = Total['date'] + ' ' + Total['time']
Total['datetime'] = pd.to_datetime(Total['datetime'])
Total['date'] = pd.to_datetime(Total['date'])
# df.index = pd.DatetimeIndex(df['datetime'])
Total= Total.fillna('')
Total['symbol'] = Total['sym_root'] + Total['sym_suffix']

In [7]:
# Include a sector column in the Total dataframe
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
use = first_table[['Symbol','GICS Sector']]
use.set_index
use = use.rename(columns={"GICS Sector": "Sector"})
use = use.set_index('Symbol')
Total['Sector'] = Total.sym_root.map(use.to_dict()['Sector'])

In [8]:
# Extraneous Code for Testing
# Tech = Total.loc[Total.Sector == 'Information Technology'].groupby(['datetime','symbol']).sum().groupby('datetime').sum()
# Tech = Tech.between_time('09:31:00', '15:59:00').sort_index()
# Tech['cum_vol'] = np.nan
# Tech = Tech.resample('60s').asfreq()
# Tech['size'].fillna(0, inplace=True)
# Tech['date'] = Tech.index.date
# Tech['time'] = Tech.index.time
# Tech['date'] = pd.to_datetime(Tech.date) 
# Tech = Tech.loc[Tech.date.isin(rel), :]
# Tech = Tech.between_time('09:31:00', '15:59:00')
# Tech['date'] = pd.to_datetime(Tech.index) 
# Tech['date'] = Tech['date'].dt.date
# Tech['date'] = pd.to_datetime(Tech.date) 
# for day in rel:
#     Tech.loc[day==Tech.date, 'cum_vol'] = Tech.loc[day== Tech.date,'size'].cumsum()
# for days in rel:
#     num = Tech['cum_vol'][Tech.date == days].values[-1]
#     Tech.loc[days==Tech.date, 'Correction'] = num
# Tech['CDF'] = Tech['cum_vol']/Tech['Correction']

In [9]:
## Takes a stock with days in the relative time period (see above)
## Output the stock with upsampling and
def clean_data_distribution(stock):
    cur = stock.between_time('09:31:00', '15:59:00').sort_index()
    cur['cum_vol'] = np.nan
    cur = cur.resample('60s').asfreq()
    cur['size'].fillna(0, inplace=True)
    cur['date'] = cur.index.date
    cur['time'] = cur.index.time
    cur['date'] = pd.to_datetime(cur.date) 
    cur = cur.loc[cur.date.isin(rel), :]
    cur = cur.between_time('09:31:00', '15:59:00')
    cur['date'] = pd.to_datetime(cur.index) 
    cur['date'] = cur['date'].dt.date
    cur['date'] = pd.to_datetime(cur.date) 
    for day in rel:
        cur.loc[day==cur.date, 'cum_vol'] = cur.loc[day== cur.date,'size'].cumsum()
    for days in rel:
        num = cur['cum_vol'][cur.date == days].values[-1]
        cur.loc[days==cur.date, 'Correction'] = num
    cur['CDF'] = cur['cum_vol']/cur['Correction']
    cur['PDF'] = cur['size']/cur['Correction']
    return cur

In [10]:
# Creates a sector dataframe with intraday volume at each minute
def get_sector_data(Sec):
    Tech = Total.loc[Total.Sector == Sec].groupby(['datetime','symbol']).sum().groupby('datetime').sum()
    Tech = Tech.between_time('09:31:00', '15:59:00').sort_index()
    return Tech

In [11]:
# Creates each of the sector dataframes and the SPY dataframe
Tech = clean_data_distribution(get_sector_data("Information Technology"))
Tech = Tech.rename(columns = {Tech.columns[5]: 'CDF_SEC', Tech.columns[6]: 'PDF_SEC'})
Fin = clean_data_distribution(get_sector_data("Financials"))
Fin = Fin.rename(columns = {Fin.columns[5]: 'CDF_SEC', Fin.columns[6]: 'PDF_SEC'})
CD = clean_data_distribution(get_sector_data("Consumer Discretionary"))
CD = CD.rename(columns = {CD.columns[5]: 'CDF_SEC', CD.columns[6]: 'PDF_SEC'})
CS = clean_data_distribution(get_sector_data("Communication Services"))
CD = CS.rename(columns = {CS.columns[5]: 'CDF_SEC', CS.columns[6]: 'PDF_SEC'})
IN = clean_data_distribution(get_sector_data("Industrials"))
IN = CD.rename(columns = {IN.columns[5]: 'CDF_SEC', IN.columns[6]: 'PDF_SEC'})
HC = clean_data_distribution(get_sector_data("Health Care"))
HC = HC.rename(columns = {HC.columns[5]: 'CDF_SEC', HC.columns[6]: 'PDF_SEC'})
NRG = clean_data_distribution(get_sector_data("Energy"))
NRG = NRG.rename(columns = {NRG.columns[5]: 'CDF_SEC', NRG.columns[6]: 'PDF_SEC'})

MAT = clean_data_distribution(get_sector_data("Materials"))
MAT = MAT.rename(columns = {MAT.columns[5]: 'CDF_SEC', MAT.columns[6]: 'PDF_SEC'})

UTIL = clean_data_distribution(get_sector_data("Utilities"))
UTIL = UTIL.rename(columns = {UTIL.columns[5]: 'CDF_SEC', UTIL.columns[6]: 'PDF_SEC'})

RE = clean_data_distribution(get_sector_data("Real Estate"))
RE = RE.rename(columns = {RE.columns[5]: 'CDF_SEC', RE.columns[6]: 'PDF_SEC'})

SPY = Total.loc[Total.symbol == 'SPY']
SPY.index = pd.DatetimeIndex(SPY['datetime'])
SPY = clean_data_distribution(SPY)
SPY = SPY.rename(columns = {SPY.columns[11]: 'CDF_CTRL', SPY.columns[12]: 'PDF_CTRL'})

In [12]:
# Produces a scatter plot for each sector using the SPY as a control
def produce_scatter(sector, sector_rep):
    plt.xlabel('SPY Index Coefficient')
    plt.ylabel('Sector Coefficient')
    plt.title("Sector Versus Market Index " + sector)
    enum = Total.loc[Total.Sector == sector].symbol.unique()
    ctrl = SPY.PDF_CTRL
    sect = sector_rep.PDF_SEC
    for scks in enum:
        try:
            cur = Total.loc[Total.symbol == scks]
            cur.index = pd.DatetimeIndex(cur['datetime'])
            cur = cur.between_time('09:31:00', '15:59:00').sort_index()
            cur = clean_data_distribution(cur)
            stock= cur.PDF
            model = pd.concat([sect,ctrl,stock],axis=1)
            result = smf.ols(formula = "stock ~ ctrl + sect",data = model).fit()
            params = result.params
            sect_coef = params[2]
            spy_coef = params[1]
            plt.scatter(spy_coef,sect_coef)
            plt.annotate(scks,(spy_coef,sect_coef),fontsize=3)
        except:
            pass
    plt.savefig(sector + 'ScatterPl.png', dpi=300)
    plt.show()

In [16]:
# produce_scatter("Health Care", HC)
# MRNA = MRNA.loc[Total.symbol == 'MRNA']
# MRNA.index = pd.DatetimeIndex(LLY['datetime'])
# MRNA = MRNA.between_time('09:31:00', '15:59:00').sort_index()
# MRNA= clean_data_distribution(MRNA)

In [ ]:
# Plot the SPY, Stock, and Sector over a fixed interval with a given lag
def minute_test(ticker,Sect, minute1,minute2,lag):
    stock = Total.loc[Total.symbol == ticker]
    stock.index = pd.DatetimeIndex(stock['datetime'])
    stock = stock.between_time('09:31:00', '15:59:00').sort_index()
    stock= clean_data_distribution(stock)
    minutes = Sect.time.unique()
    avg_stock = pd.Series(dtype='float64')
    avg_Sector = pd.Series(dtype='float64')
    avg_SPY = pd.Series(dtype='float64')
    for minute in minutes:
        avg_stock.loc[minute] = stock.loc[stock.index.time == minute, 'PDF'].mean()
        avg_Sector.loc[minute] = Sect.loc[Sect.index.time == minute, 'PDF_SEC'].mean()
        avg_SPY.loc[minute] = SPY.loc[SPY.index.time == minute, 'PDF_CTRL'].mean()
    plt.plot(avg_stock.reset_index(drop=True)[minute1 - 1:minute2],label=ticker)
    plt.plot(avg_SPY.reset_index(drop=True)[minute1 - 1:minute2],label = "SPY")
    plt.plot(avg_Sector.reset_index(drop=True).shift(periods=lag)[minute1 - 1:minute2],label = "Sector")
    plt.legend()
    plt.title("PDF Curves for Sector with lag = " + str(-lag) + " and " + ticker)
    plt.xlabel("Minute")
    plt.ylabel("Percentage of Volume Traded")
    plt.savefig(str(minute2 - minute1) +'_Minutes_' + ticker + 'lag=' + str(-lag) + '.png', dpi=300)
    plt.show()

In [26]:
# Perform a regression on the stock volume based on stock and sector volumes from previous minutes (lag)
def lag_regression(ticker,Sect,lag):
    stock = Total.loc[Total.symbol == ticker]
    stock.index = pd.DatetimeIndex(stock['datetime'])
    stock = stock.between_time('09:31:00', '15:59:00').sort_index()
    stock= clean_data_distribution(stock)
    stck = stock.PDF[::-1 * lag]
    ctrl = stock.shift(periods = -lag).PDF[::-1 * lag]
    sect=Sect.PDF_SEC.shift(periods = -lag)[::-1 * lag]
    model = pd.concat([sect,stck,ctrl],axis=1)
    result = smf.ols(formula = "stck ~ sect + ctrl",data = model).fit()
    return result

In [38]:
# Perform a regression on the stock volume based on sector volumes from previous minutes (lag)
def lag_regression_without(ticker,Sect,lag):
    stock = Total.loc[Total.symbol == ticker]
    stock.index = pd.DatetimeIndex(stock['datetime'])
    stock = stock.between_time('09:31:00', '15:59:00').sort_index()
    stock= clean_data_distribution(stock)
    stck = stock.PDF[::-1 * lag]
    ctrl = stock.shift(periods = -lag).PDF[::-1 * lag]
    sect=Sect.PDF_SEC.shift(periods = -lag)[::-1 * lag]
    model = pd.concat([sect,stck,ctrl],axis=1)
    result = smf.ols(formula = "stck ~ ctrl",data = model).fit()
#     print(result.summary())
    return result

In [30]:
# Scatterplot of the coefficients from the lag_regression method across all stocks in a given sector
# We use the try function since not all of the stocks in the current S&P500 existed during the testing window
def produce_lag_scatter(sector, sector_rep, lag):
    plt.xlabel('Lag Term for Stock')
    plt.ylabel('Lag Term for Sector')
    plt.title("Sector Versus Market Index " + sector + 'with lag = ' + str(lag))
    enum = Total.loc[Total.Sector == sector].symbol.unique()
    for scks in enum:
        try:
            result = lag_regression(scks,sector_rep,lag)
            params = result.params
            sect_coef = params[2]
            ctrl_coef = params[1]
            plt.scatter(ctrl_coef,sect_coef)
            plt.annotate(scks,(ctrl_coef,sect_coef),fontsize=3)
        except:
            pass
    plt.savefig(sector + 'Lag' + str(lag)+ '_regression.png', dpi=300)
    plt.show()